In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
from sympy import *
import pickle
import datetime
from tqdm import tqdm
import scipy.constants
import time

from flowline_ode.plots_setup import *
from flowline_ode.sia_model import *
from flowline_ode.finite_differences import *
from flowline_ode.ode_solve import *

In [ ]:
t_start_notebook = time.time()

In [ ]:
# Domain size
domain_x = 50000 # meters # 100 km for all other examples, 200 km for the new one
domain_z = 3000 # meters

# Grids for when discretization is needed
dx = 100
dz = 25
xs = np.arange(0, domain_x, dx)
zs = np.arange(0, domain_z, dz)

# Sympy symbolic variables
x = sympy.symbols('x', real=True, positive=True)
z = sympy.symbols('z', real=True, positive=True)

Two examples:

* Example A: Sloped but parallel layers
* Example B: Plug flow but variation in layer slope with depth

In [ ]:
# Setup geometry and error types for each of the two examples
surface_sym = [2700 - (500/domain_x)*x, 0*x + 2500]
basal_velocity = [0*x, 0*x + 10/scipy.constants.year]
error_types = ['difference', 'difference'] # Error types are how the error in vertical velocity will be plotted:
                                        # 'percent' is error as a fraction of the true value,
                                        # 'difference' is zero slope approx minus true

surface = [lambdify_and_vectorize_if_needed(x, s) for s in surface_sym]

ds_dx_sym = [sympy.diff(s, x) for s in surface_sym]

u = []
w = []
du_dx = []

for idx in range(len(surface)):
    u_tmp, w_tmp, du_dx_tmp = sia_model(x, z, surface_sym[idx], ds_dx_sym[idx],
                        basal_velocity_sym=basal_velocity[idx], n=3)
    
    u.append(u_tmp)
    w.append(w_tmp)
    du_dx.append(du_dx_tmp)

In [ ]:
# Layers

xs_layers = np.linspace(0, domain_x, 100) # don't need tight grid spacing for smooth layers

layers_t0 = [[] for _ in range(len(surface))]

# Example A
# Manually specified layers -- Zero slope error example A (dtan(alpha)/dz = 0)
layer_base_fn = lambda x: ((2200-1700) / (1 + np.exp(0.0002*(x-(2*domain_x/3))))) + 1600
for idx, start_offset in enumerate(np.arange(0, 1500, 150)):
    layer_start_fn = lambda x: (1 + 0*idx)*layer_base_fn(x) - start_offset
    layer = advect_layer(lambdify_and_vectorize_if_needed((x, z), u[0]),
                         lambdify_and_vectorize_if_needed((x, z), w[0]),
                         xs_layers, layer_start_fn, [scipy.constants.year * 0])
    layers_t0[0].append(layer[-1])

# Example B
# Manually specified layers -- Zero slope error example B (du/dz = 0)
for idx, start_offset in enumerate(np.arange(2300, 1200, -120)):
    def layer_fn(x):
        x = x - domain_x/2
        l = -2500 * idx/10 * np.exp(-1 / (1-(x/15e3)**2))
        l[np.abs(x) >= 15e3] = 0
        l += start_offset
        return l
    layer = advect_layer(lambdify_and_vectorize_if_needed((x, z), u[1]),
                         lambdify_and_vectorize_if_needed((x, z), w[1]),
                         xs_layers, layer_fn, [scipy.constants.year * 0])
    layers_t0[1].append(layer[-1])

# Advect both sets of layers by 1 year and create finite difference functions
layer_dl_dx = []
layer_dl_dt = []

xs_layers_t1 = np.arange(0, domain_x, 10)
layers_t1 = [[] for _ in range(len(surface))]
for idx in range(len(surface)):
    for layer in layers_t0[idx]:
        layers_t1[idx].append(advect_layer(lambdify_and_vectorize_if_needed((x, z), u[idx]),
                                           lambdify_and_vectorize_if_needed((x, z), w[idx]),
                                           xs_layers_t1, layer, [scipy.constants.year * 1])[-1])
    
    # Create finite difference approximation functions for the simulated layer measurements
    layer_dl_dx_tmp, layer_dl_dt_tmp, _, _ = create_layer_finite_difference_fns(layers_t0[idx], layers_t1[idx])

    layer_dl_dx.append(layer_dl_dx_tmp)
    layer_dl_dt.append(layer_dl_dt_tmp)

In [ ]:
X, Z = np.meshgrid(xs, zs)
below_surface_mask = []
# Below surface masks
for idx in range(len(surface)):
    below_surface = (Z < surface[idx](X))
    below_surface_mask.append(np.ones_like(X, dtype=float))
    below_surface_mask[idx][~below_surface] = np.nan

U = [[] for _ in range(len(surface))]
W = [[] for _ in range(len(surface))]

dl_dx_grid = [[] for _ in range(len(surface))]
dl_dt_grid = [[] for _ in range(len(surface))]
dw_dz_zeroslope_grid = [[] for _ in range(len(surface))]
     
for idx in range(len(surface)):
    U_tmp, W_tmp = uw_on_grid(xs, zs, u[idx], x, z,
                              lambdify_and_vectorize_if_needed(x, basal_velocity[idx]),
                              domain_z)
    U[idx] = U_tmp
    W[idx] = W_tmp

    layer_dl_dx_tmp, layer_dl_dt_tmp, _, _ = create_layer_finite_difference_fns(layers_t0[idx], layers_t1[idx])

    interp_x = np.zeros((len(layers_t0[idx]), len(xs)))
    interp_z = np.zeros_like(interp_x)
    interp_dl_dt = np.zeros_like(interp_x)
    interp_dl_dx = np.zeros_like(interp_x)
    for layer_idx in range(len(layers_t0[idx])):
        interp_x[layer_idx, :] = xs
        interp_z[layer_idx, :] = layers_t0[idx][layer_idx](xs)
        interp_dl_dt[layer_idx, :] = layer_dl_dt_tmp(xs, layer_idx)
        interp_dl_dx[layer_idx, :] = layer_dl_dx_tmp(xs, layer_idx)

    dl_dt_grid[idx] = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dt.flatten(), (X, Z), method='linear')
    dl_dx_grid[idx] = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dx.flatten(), (X, Z), method='linear')
    dw_dz_zeroslope_grid[idx] = np.gradient(dl_dt_grid[idx], zs, axis=0)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 5), sharex=True, sharey=True)

plot_labels_left = ['(a)', '(b)']

# Plot vertical velocity and layers on left
W_max = np.max([np.nanmax(W[idx]*below_surface_mask[idx]) for idx in range(len(surface))]) * scipy.constants.year

pcm_vv = None
pcm_err = None

for idx in range(len(surface)):
    ax = axs[idx, 0]

    # Between layers mask
    between_layers_mask = np.ones_like(below_surface_mask[idx])
    between_layers_mask[Z > layers_t0[idx][0](xs)] = np.nan
    between_layers_mask[Z < layers_t0[idx][-1](xs)] = np.nan

    # Vertical velocity
    vv = scipy.constants.year*W[idx]*below_surface_mask[idx]
    pcm_vv = ax.pcolormesh(X/1e3, Z, vv, cmap='YlOrRd', vmin=0, vmax=W_max)

    

    # Plot error (zero slope aprrox vertical velocity minus true vertical velocity)

    ax = axs[idx, 1]
    err = between_layers_mask*(dl_dt_grid[idx] - scipy.constants.year*W[idx]*below_surface_mask[idx])

    pcm_err = ax.pcolormesh(X/1e3, Z, err, cmap='coolwarm', vmin=-1.5, vmax=1.5)

    # Overlay layers and surface
    for ax in axs[idx, :]:
        for layer in layers_t0[idx]:
            ax.plot(xs_layers/1e3, layer(xs_layers), 'k--', linewidth=0.5)
        ax.plot(xs_layers/1e3, surface[idx](xs_layers), 'k')

        ax.set_xlabel('x [km]')
        ax.set_ylabel('z [m]')
        ax.set_ylim([0, domain_z])
        ax.set_xlim([0, domain_x/1e3])

subfigure_labels = ['(a)', '(b)', '(c)', '(d)']
for idx, ax in enumerate(axs.flatten()):
    ax.set_title(subfigure_labels[idx], loc='left', fontweight='bold')

fig.colorbar(pcm_vv, ax=axs[:, 0], label='Vertical Velocity [m/yr]')
fig.colorbar(pcm_err, ax=axs[:, 1], label='Vertical Velocity Error [m/yr]\nwith zero slope approximation')

# Add layer line legend
import matplotlib.lines as mlines
layer_line = mlines.Line2D([], [], linewidth=0.5, linestyle='--', color='k', label='Layers')
axs[1,0].legend(handles=[layer_line], bbox_to_anchor=(0.3, -0.15))

fig.savefig('figures/zero-slope-errors.png', dpi=500, bbox_inches="tight")